In [ ]:
# Import relevant libraries
import os
import pandas as pd

from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT

import numpy as np

from SALib.sample import saltelli

from comparative_gsa.sample_input_space import sample_input_space

import json

from ModularCirc import BatchRunner

from comparative_gsa.simulate_data import simulate_data
from comparative_gsa.calculate_output_features import calculate_output_features

from SALib.analyze.sobol import analyze
import matplotlib.pyplot as plt

In [ ]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

n_samples = 245760

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'


In [ ]:
# Load input_544_samples.csv
input_csv = os.path.join(simulation_out_path, f"input_samples_{n_samples}.csv")
X = pd.read_csv(input_csv)
print("Loaded input samples:", X.shape)

# Load simulation_summary.csv
summary_csv = os.path.join(simulation_out_path, "simulations_summary.csv")
Y = pd.read_csv(summary_csv)
print("Loaded simulation summary:", Y.shape)

In [ ]:
output_feature = "p_ao_max"  # Change to any column name in Y
Y_feature = Y[output_feature].values  # .values converts to numpy array.
Y_feature.shape

In [ ]:
relevant_columns = ['ao.r',
 'ao.c',
 'art.r',
 'art.c',
 'ven.r',
 'ven.c',
 'av.r',
 'mv.r',
 'la.E_pas',
 'la.E_act',
 'la.k_pas',
 'lv.E_pas',
 'lv.E_act',
 'lv.k_pas']

problem = {
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds' : X[relevant_columns].describe().loc[['min', 'max']].T.values
}

In [ ]:
# Instead, load the pre-existing problem definition and saltelli samples
problem_path = '../outputs/simulations/output_245760_samples_parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas/problem.pkl'

# Load the problem pickle file
import pickle
import pandas as pd
with open(problem_path, 'rb') as f:
    problem = pickle.load(f)

problem

In [ ]:
# Do the sobol_analyse for GSA
sobol_indices = analyze(problem, Y_feature, calc_second_order=True)

In [ ]:
sobol_indices

In [ ]:
from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from autoemulate.core.sensitivity_analysis import _sobol_results_to_df 

In [ ]:
results = {
    output_feature: sobol_indices
}

In [ ]:
sobol_df = _sobol_results_to_df(results)

In [ ]:
# Due to a bug in autoemulate plotting, we must swap ST and S1 rows.

# Get the indices of rows where index == 'ST'
mask_st = sobol_df['index'] == 'ST'
mask_s1 = sobol_df['index'] == 'S1'

# For those rows, change the index to be 'S1'
sobol_df.loc[mask_st, 'index'] = 'S1'

# For those rows, change the index to be 'ST'
sobol_df.loc[mask_s1, 'index'] = 'ST'

In [ ]:
figsize = (9, 5)

SensitivityAnalysis.plot_sobol(sobol_df, index="S1", figsize=figsize) 

In [ ]:
figsize = (9, 5)

SensitivityAnalysis.plot_sobol(sobol_df, index="ST", figsize=figsize) 

In [ ]:
# Create directories for saving plots if they don't exist
plots_dir = os.path.join(simulation_out_path, 'plots', f'{output_feature}_{n_samples}_samples')
os.makedirs(plots_dir, exist_ok=True)

figsize = (9, 5)

# Plot and save S1
fig_S1 = SensitivityAnalysis.plot_sobol(sobol_df, index="S1", figsize=figsize)
if fig_S1 is None:
    # If the function does not return a fig, get the current figure instead
    fig_S1 = plt.gcf()
fig_S1.savefig(os.path.join(plots_dir, f'S1_{output_feature}_{n_samples}_samples.png'), 
               dpi=300, bbox_inches='tight')
fig_S1.savefig(os.path.join(plots_dir, f'S1_{output_feature}_{n_samples}_samples.pdf'), 
               bbox_inches='tight')
plt.close(fig_S1)  # Close the figure to free memory

print(f"S1 plot saved to: {plots_dir}")

# Plot and save ST
fig_ST = SensitivityAnalysis.plot_sobol(sobol_df, index="ST", figsize=figsize)
if fig_ST is None:
    fig_ST = plt.gcf()
fig_ST.savefig(os.path.join(plots_dir, f'ST_{output_feature}_{n_samples}_samples.png'), 
               dpi=300, bbox_inches='tight')
fig_ST.savefig(os.path.join(plots_dir, f'ST_{output_feature}_{n_samples}_samples.pdf'), 
               bbox_inches='tight')
plt.close(fig_ST)

print(f"ST plot saved to: {plots_dir}")

In [ ]:
# Plot a histogram of y_pred_np
plt.hist(Y_feature, bins=1000, alpha=0.7)
plt.xlabel('Predicted Values')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Values')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 18))
axes = axes.flatten()

for i, param_name in enumerate(X[relevant_columns].columns):
    axes[i].scatter(Y_feature, X[relevant_columns][param_name], alpha=0.1, s=0.1)
    axes[i].set_xlabel(f'{output_feature} emulated')
    axes[i].set_ylabel(param_name)
    axes[i].set_title(param_name)

# Add a super title
plt.suptitle(f'{output_feature} corr with parameters', fontsize=16)

plt.tight_layout()
plt.show()


In [ ]:
problem['bounds'][-1]